# Process 5p and Intron sequencing data (Run on Genius)

In [ ]:
cd /staging/leuven/stg_00002/lcb/lcb_projects/CH1/5p_intron/counting/

## 00. Combine fastq from different sequencing runs

In [ ]:
# Path to the different runs
HiSeq_1902="/staging/leuven/stg_00002/lcb/ngs_runs/HiSeq4000_20190222/Demultiplexed/{}*L00*"
NextSeq="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20190403/Demultiplexed/{}*R1*"
HiSeq_1906="/staging/leuven/stg_00002/lcb/ngs_runs/HiSeq4000_20190620/Demulitplexed/{}*"
NovaSeq="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20190627/Demultiplexed/{}*R1*"
NovaSeq_201126="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201126/Demultiplexed/{}*R1*"
NextSeq_20210330="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq2000_20210330/Demultiplexed/{}*R1*"

In [ ]:
parallel \
"zcat $NextSeq_20210330 | gzip > 00.combined_raw/{}_c.fastq.gz &&
   echo -e \"Combined number of reads for {}: \"\$(( \$(zcat 00.combined_raw/{}_c.fastq.gz | wc -l) / 4 )) \"\\n\"Duplicated read IDs: \$(zcat 00.combined_raw/{}_c.fastq.gz | grep '^@' | sort | uniq -d | wc -l)" \
   ::: $(cat 00.combined_raw/CHEQseq_5p_intron_names.txt)
   

## 01. Make FastQC

In [ ]:
module load FastQC/0.11.8-Java-1.8.0_162
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
fastqc 00.combined_raw/${line}_c.fastq.gz -o 01.fastqc
done

## 02.1 Clean reads

In [ ]:
# Remove 5' adapter and trim read length to BC length
module load cutadapt/1.18-foss-2018a-Python-3.7.0
 # 5' samples
grep '5_prime' 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do

cutadapt -g ACTCGAGCCGAGCAGGCGCGCCGATC...GGACCAG -e 0.2 -l 17 -m 17 -j 6 --discard-untrimmed \
         -o 02.cleaned_reads/${line}c.fastq.gz \
         00.combined_raw/${line}_c.fastq.gz
done

In [ ]:
 # Intron samples
grep 'Intron' 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
cutadapt -g GTCCACTCCCAGGTGGCGCGCCCTAG...TGCAGG -e 0.2 -l 18 -m 18 -j 6 --discard-untrimmed \
         -o 02.cleaned_reads/${line}c.fastq.gz \
         00.combined_raw/${line}_c.fastq.gz
done

## 02.2 Q30 filtering

In [ ]:
# Filtering out reads with average phred score < 30
module load fastp/0.20.0-foss-2018a
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
fastp -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 \
      -o 02.cleaned_reads/${line}_ccq30.fastq.gz \
      -i 02.cleaned_reads/${line}c.fastq.gz
done

In [ ]:
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
echo ${line}
echo $(( $(zcat 02.cleaned_reads/${line}_ccq30.fastq.gz | wc -l) / 4 ))
done

## 03. Sequencing saturation

In [ ]:
# Extract DNA sequences from fastq
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
zcat 02.cleaned_reads/${line}_ccq30.fastq.gz | sed -n '2~4p' > 03.sequencing_saturation/${line}.txt
done

In [ ]:
# Subsample, count total and unique reads
start=`date +%s`

cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
file="03.sequencing_saturation/${line}.txt"
nreads=$(cat $file | wc -l)
 # Start by removing parallel instructions file
rm 03.sequencing_saturation/parallel.txt
 # Remove any previoulsy created sat values for that file
rm ${file%_*}_sat.txt
    for i in $(seq 0 200000 $nreads)
    do
     # Print number of total and unique reads for every subsamples
    echo "echo $i \$(shuf -n $(($i)) $file | sort | uniq | wc -l) >> ${file%_*}_sat.txt" \
    >> 03.sequencing_saturation/parallel.txt
    done
cat 03.sequencing_saturation/parallel.txt | parallel -j 16
 # Put values in numerical order
cat ${file%_*}_sat.txt | sort -g > ${file%_*}_satsort.txt && mv ${file%_*}_satsort.txt ${file%_*}_sat.txt
done

end=`date +%s`
runtime=$((end-start))
echo $runtime

## 10. Make BC count matrix

In [ ]:
# Generate BC count matrix for each samples
parallel \
"cat 03.sequencing_saturation/{}.txt | sort | uniq -c | awk -F \" \" '{print \$2\"\t\"\$1}' \
   > 10.bc_count/{}_count.txt" ::: $(cat 00.combined_raw/CHEQseq_5p_intron_names.txt)
 # Count number of unique BC per sample
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
echo ${file#*/}
echo 10.bc_count/${line}_count.txt && cat 10.bc_count/${line}_count.txt | wc -l
done

In [ ]:
# Combine Enhancer and BC
start=`date +%s`
 # 5' samples
enh_coupling_5p="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/EA/CHEQ-seq_pSA073/data/2917_DMauduit_PacBio/201.coupling/pacbio_3passes_5prime_plasmid_enhancer_barcode_and_negctrl_barcode_minQ30_final_list.tsv"
grep '5_prime' 00.combined_raw/CHEQseq_5p_intron_names.txt | while read name
do
rm 10.bc_count/parallel_combine2.txt
rm 10.bc_count/${name}_final*
cat 10.bc_count/${name}_count.txt | while read line
    do
    echo "echo -e \$(grep ${line%%$'\t'*} ${enh_coupling_5p} | awk '{print \$1}')\"\t\"${line%%$'\t'*}\"\t\"${line##*$'\t'} >> 10.bc_count/${name}_final_NEG.txt" \
    >> 10.bc_count/parallel_combine2.txt
    done
echo "Processing ${name}..."
cat 10.bc_count/parallel_combine2.txt | parallel -j 30  
sed '/^\t/ d' < 10.bc_count/${name}_final_NEG.txt > 10.bc_count/${name}_final_clean_NEG.txt
rm 10.bc_count/${name}_final_NEG.txt
done
end=`date +%s`
echo $((end-start))

In [ ]:
start=`date +%s`
 # Intron samples
enh_coupling_intron="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/EA/CHEQ-seq_pSA073/data/2917_DMauduit_PacBio/201.coupling/pacbio_3passes_intron_plasmid_enhancer_barcode_and_negctrl_barcode_minQ30_final_list.tsv"
grep 'Intron' 00.combined_raw/CHEQseq_5p_intron_names.txt | while read name
do
rm 10.bc_count/parallel_combine2.txt
rm 10.bc_count/${name}_final*
cat 10.bc_count/${name}_count.txt | while read line
    do
    echo "echo -e \$(grep ${line%%$'\t'*} ${enh_coupling_intron} | awk '{print \$1}')\"\t\"${line%%$'\t'*}\"\t\"${line##*$'\t'} >> 10.bc_count/${name}_final_NEG.txt" \
    >> 10.bc_count/parallel_combine2.txt
    done
echo "Processing ${name}..."
cat 10.bc_count/parallel_combine2.txt | parallel -j 30    
sed '/^\t/ d' < 10.bc_count/${name}_final_NEG.txt > 10.bc_count/${name}_final_clean_NEG.txt
rm 10.bc_count/${name}_final_NEG.txt
done
end=`date +%s`
echo $((end-start))

In [ ]:
# Count number of BC associated to an enhancer
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
echo -e "${line}\t\t$(cat 10.bc_count/${line}_final_clean_NEG.txt | wc -l)"
done

In [ ]:
# Count number of BC per enhancer
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
cat 10.bc_count/${line}_final_clean_NEG.txt | awk '{print $1}' | sort | uniq -c > 10.bc_count/10.1.Summary/${line}_summary.txt
cat 10.bc_count/${line}_final_clean_NEG.txt | awk '{print $1}' | sort | uniq -c | wc -l >> 10.bc_count/10.1.Summary/${line}_summary.txt
done

In [ ]:
# Median number of reads per BC
cat 00.combined_raw/CHEQseq_5p_intron_names.txt | while read line
do
middle=$(($(cat 10.bc_count/${line}_final_clean_NEG.txt | wc -l)/2))
echo ${line}
awk '{print $NF}' 10.bc_count/${line}_final_clean_NEG.txt | sort -g | sed -n ${middle}p
done